In [2]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/reg_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-07-28 12:30:10.241722


## Regression: Comparing target's same MSA cites to different MSA cites

### 1. Create new regression sample using only target's similarities

In [20]:
ts = pd.read_pickle("DataStore/2018-07-P2/targ_samp_cite_sim_0717.pkl")

# Get rid of 2010+
ts = ts.loc[ts["tp_gyear"] < 2010]
# Get rid of cp columns
ts = ts[[c for c in ts.columns if "_cp_" not in c]]

# Add year group
def get_year_group(x):
    if x in range(1975,1980):
        yg = "1975-80"
    elif x in range(1980,1985):
        yg = "1980-85"
    elif x in range(1985, 1990):
        yg = "1985-90"
    elif x in range(1990,1995):
        yg = "1990-95"
    elif x in range(1995,2000):
        yg = "1995-00"
    elif x in range(2000,2005):
        yg = "2000-05"
    elif x in range(2005,2010):
        yg = "2005-10"
    elif x in range(2010, 2015):
        yg = "2010-15"
    else:
        yg = np.nan
    return yg

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

ts["year_group"] = ts["tp_gyear"].apply(get_year_group_10)

# Scale
# eps = 0.01
# dv_min = 0.7
# def scale_docvecs(x):
#     scaled = ((x+dv_min)/(1+dv_min))*(1-eps)+eps
#     return scaled
# def scale_ldavecs(x):
#     scaled = x*(1-eps)+eps
#     return scaled

sim_cols = [c for c in ts.columns if "sim_" in c]
# for c in sim_cols:
#     if "docvecs" in c:
#         ts[c] = ts[c].apply(scale_docvecs)
#     else:
#         ts[c] = ts[c].apply(scale_ldavecs)

# Normed values
for c in sim_cols:
    ts["norm_{0}".format(c)] = np.nan
    ts.loc[ts[c].notnull(), "norm_{0}".format(c)] = \
    scaler.fit_transform(ts.loc[ts[c].notnull(), c].values.reshape(-1,1))

In [21]:
ts.head()

,tp,cp,tp_primclass,tp_gyear,tp_inv_msa,cp_gyear,cp_inv_msa,tp_match_5,cp_match_5,tp_match_10,...,sim_docvecs_tp_cite_msa_diff_10,year_group,norm_sim_ldavecs_tp_cite_msa_match_5,norm_sim_ldavecs_tp_cite_msa_diff_5,norm_sim_ldavecs_tp_cite_msa_match_10,norm_sim_ldavecs_tp_cite_msa_diff_10,norm_sim_docvecs_tp_cite_msa_match_5,norm_sim_docvecs_tp_cite_msa_diff_5,norm_sim_docvecs_tp_cite_msa_match_10,norm_sim_docvecs_tp_cite_msa_diff_10
0,5241648,5241671,1.0,1993,"San Jose-Sunnyvale-Santa Clara, CA",1993,"San Diego-Carlsbad-San Marcos, CA",0.200000,0.064516,0.285714,...,0.312115,1985-95,-0.902104,0.430012,-0.024628,0.313008,-0.730488,-0.096243,0.399949,0.278223
1,6665662,6665663,1.0,2003,"San Francisco-Oakland-Fremont, CA",2003,"San Jose-Sunnyvale-Santa Clara, CA",0.000000,0.500000,0.145833,...,0.248978,1995-05,NaN,0.016959,-0.687718,-0.167941,NaN,-0.530843,-0.828639,-0.276915
2,5797117,5797128,1.0,1998,"Los Angeles-Long Beach-Santa Ana, CA",1998,"Colorado Springs, CO",0.000000,0.000000,0.000000,...,0.388110,1995-05,NaN,0.210453,NaN,0.277285,NaN,0.842367,NaN,0.946418
3,7502777,7502779,1.0,2009,"San Jose-Sunnyvale-Santa Clara, CA",2009,"Austin-Round Rock-San Marcos, TX",0.000000,0.000000,0.000000,...,0.395416,2005-15,NaN,0.723882,NaN,0.817262,NaN,0.902428,NaN,1.010653
4,7599939,7599941,1.0,2009,"San Jose-Sunnyvale-Santa Clara, CA",2009,"Los Angeles-Long Beach-Santa Ana, CA",0.166667,0.150000,0.250000,...,0.374581,2005-15,NaN,1.109816,NaN,1.223151,NaN,0.731143,NaN,0.827465


In [22]:
ts1 = ts[['tp', 'tp_primclass', 'tp_gyear', 'tp_inv_msa','sim_ldavecs_tp_cite_msa_diff_10',
          'sim_docvecs_tp_cite_msa_diff_10', 'norm_sim_ldavecs_tp_cite_msa_diff_10',
          'norm_sim_docvecs_tp_cite_msa_diff_10',
         "year_group", ]]
ts1.columns = ['tp', 'tp_primclass', 'tp_gyear', 'tp_inv_msa', 
       'sim_ldavecs_tp_cite_10','sim_docvecs_tp_cite_10',
               'norm_sim_ldavecs_tp_cite_10','norm_sim_docvecs_tp_cite_10',
               "year_group"]
ts1["inv_msa_match"] = False
ts2 = ts[['tp', 'tp_primclass', 'tp_gyear', 'tp_inv_msa', 'sim_ldavecs_tp_cite_msa_match_10',
        'sim_docvecs_tp_cite_msa_match_10', 'norm_sim_ldavecs_tp_cite_msa_match_10',
        'norm_sim_docvecs_tp_cite_msa_match_10',
        "year_group"]]
ts2.columns = ['tp', 'tp_primclass', 'tp_gyear', 'tp_inv_msa', 
       'sim_ldavecs_tp_cite_10', 'sim_docvecs_tp_cite_10',
               'norm_sim_ldavecs_tp_cite_10','norm_sim_docvecs_tp_cite_10',
              "year_group"]
ts2["inv_msa_match"] = True
ts = ts1.append(ts2, ignore_index = True)
del(ts1, ts2)

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
ts.to_pickle(pathdir+"tp_cite_loc_sim_0727.pkl")

_____
Start here

## Regressions

In [3]:
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
ts = pd.read_pickle(pathdir+"tp_cite_loc_sim_0727.pkl")
ts["tp_primclass"] = ts["tp_primclass"].astype(int)

# # Remove outliers
# print(len(ts))
# ts = ts.loc[ts['norm_sim_docvecs_tp_cite_10'] >= -4]
# print(len(ts))

In [5]:
dm = "norm_sim_docvecs_tp_cite_10"
display(ts.loc[ts["inv_msa_match"] == True, [dm, "year_group"]].groupby("year_group").mean())
display(ts.loc[ts["inv_msa_match"] == False, [dm, "year_group"]].groupby("year_group").mean())

,norm_sim_docvecs_tp_cite_10
year_group,
1975-85,0.164911
1985-95,0.061851
1995-05,-0.038389
2005-15,-0.067204


,norm_sim_docvecs_tp_cite_10
year_group,
1975-85,0.119624
1985-95,0.047966
1995-05,-0.033517
2005-15,-0.082328


In [6]:
dm = "sim_docvecs_tp_cite_10"
display(ts.loc[ts["inv_msa_match"] == True, [dm, "year_group"]].groupby("year_group").mean())
display(ts.loc[ts["inv_msa_match"] == False, [dm, "year_group"]].groupby("year_group").mean())

,sim_docvecs_tp_cite_10
year_group,
1975-85,0.348148
1985-95,0.332560
1995-05,0.317398
2005-15,0.313040


,sim_docvecs_tp_cite_10
year_group,
1975-85,0.294077
1985-95,0.285927
1995-05,0.276660
2005-15,0.271109


In [14]:
r_f = "DataStore/2018-07-P2/Reg0726/reg_model_0726.pkl"
regs = pickle.load(open(r_f, "rb"))

# Similarity regressions
cit_reg = []
for c in ['sim_ldavecs_tp_cite_10', 'sim_docvecs_tp_cite_10']:
    cit_reg.extend([
        "{0} ~ C(inv_msa_match) + C(tp_gyear)".format(c),
        "{0} ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)".format(c),
        "norm_{0} ~ C(inv_msa_match) + C(tp_gyear)".format(c),
        "norm_{0} ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)".format(c),
    ])
    regs["JTH Targ"] = pd.Series(cit_reg)
    
    regs["JTH Targ model names"] = pd.Series([
        "Sim LDAVecs - 10 yrs, Year FE",
        "Sim LDAVecs - 10 yrs, PC FE",
        "N Sim LDAVecs - 10 yrs, Year FE",
        "N Sim LDAVecs - 10 yrs, PC FE",
        "Sim DocVecs - 10 yrs, Year FE",
        "Sim DocVecs - 10 yrs, PC FE",
        "N DocVecs - 10 yrs, Year FE",
        "N DocVecs - 10 yrs, PC FE",
    ])
    
    regs["JTH Targ Norm"] = regs["JTH Targ"].loc[[2,3,6,7]]
pickle.dump(regs, open(r_f, "wb"))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [15]:
info_dict = {'$N$':lambda x: "{0:d}".format(int(x.nobs)),
'Adjusted $R^2$':lambda x: "{:.2f}".format(x.rsquared_adj)}
def get_fit(formula, grouped_data, group_col, cov_type, return_fit = False):
    summ = []
    tables = {}
    
    # Remove missing values used in formula
    col_used = re.findall('\((.*?)\)',formula)
    grouped_data = grouped_data.dropna(how="any", subset=col_used) # Should take care of gyear > 1980 issues
    
    # Group and then get results
    grouped_data = grouped_data.groupby(group_col)
    
    for n,g in grouped_data:
        try:
            fit = smf.ols(formula = formula, data = g, missing="drop").fit(cov_type=cov_type)
            # Get results tables
            tables[n] = fit.summary2().tables
            # Append results
            summ.append(fit)
        except:
            pass
    # Get full results output
    # Dataframe of full results
    res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    
    # Get partial results
    # 1. Get relevant variables from index of full results
    regressors = [v for v in res_no_stars.index.unique() if ("sim_" in v) | ("match" in v)]
    # 2. Make sure regressors come last
    regressors = regressors+["Intercept"]
    # 3. Get results with regressors
    part_res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    part_res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    
    # 4. Get index of where Intercept is and add 2 (to include standard error)
    last_ind = list(part_res_stars.index).index("Intercept")+2
    
    # 5. Get partial results
    part_res_no_stars = pd.concat([part_res_no_stars.iloc[:last_ind], part_res_no_stars.iloc[-2::]])
    part_res_stars = pd.concat([part_res_stars.iloc[:last_ind], part_res_stars.iloc[-2::]])
    
    if return_fit == True:
        return summ, tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    else:
        return tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars

In [16]:
print("started")
print(datetime.datetime.now())
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
# 1. Load file
rs = pd.read_pickle(pathdir+"tp_cite_loc_sim_0727.pkl")
print(len(rs))
# 3. Define output
samp_out = {}
formulas = list(regs["JTH Targ"])
formulas_ind = list(regs["JTH Targ"].index)
cov = "HC1"
for i, j in zip(formulas_ind, formulas):
    print(i)
    print(j)
    print(datetime.datetime.now())
    try:
        out = get_fit(j, rs, "year_group", cov, return_fit = False)
        samp_out[i] = {}
        samp_out[i]["model"] = j
        samp_out[i]["tables"] = out[0]
        samp_out[i]["res_no_stars"] = out[1]
        samp_out[i]["res_stars"] = out[2]
        samp_out[i]["part_res_no_stars"] = out[3]
        samp_out[i]["part_res_stars"] = out[4]
    except Exception as e:
        logging.exception("error here")
        pass
    print("finished")
    print(datetime.datetime.now())
pickle.dump(samp_out, open(pathdir+"JTH_tp_cite_loc_res_0726.pkl", "wb"))    

started
2018-07-28 20:46:28.852640
480612
0
sim_ldavecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_gyear)
2018-07-28 20:46:28.930171
finished
2018-07-28 20:46:30.734408
1
sim_ldavecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)
2018-07-28 20:46:30.734461
finished
2018-07-28 20:46:50.757369
2
norm_sim_ldavecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_gyear)
2018-07-28 20:46:50.757422
finished
2018-07-28 20:46:52.555127
3
norm_sim_ldavecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)
2018-07-28 20:46:52.555190
finished
2018-07-28 20:47:12.471812
4
sim_docvecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_gyear)
2018-07-28 20:47:12.471876
finished
2018-07-28 20:47:14.242821
5
sim_docvecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)
2018-07-28 20:47:14.242871
finished
2018-07-28 20:47:34.633061
6
norm_sim_docvecs_tp_cite_10 ~ C(inv_msa_match) + C(tp_gyear)
2018-07-28 20:47:34.633127
finished
2018-07-28 20:47:36.442255
7
norm_sim_docvecs_tp_cite_10 ~ C(inv_m

### Check data

In [6]:
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
ts = pd.read_pickle(pathdir+"tp_cite_loc_sim_0727.pkl")
ts["tp_primclass"] = ts["tp_primclass"].astype(int)
ts.loc[(ts["inv_msa_match"] == True), ['sim_ldavecs_tp_cite_10', 'sim_docvecs_tp_cite_10']].describe()

,sim_ldavecs_tp_cite_10,sim_docvecs_tp_cite_10
count,57897.000000,57897.000000
mean,0.549620,0.323205
std,0.221865,0.151252
min,0.001521,-0.552402
25%,0.391916,0.224268
50%,0.554814,0.313963
75%,0.720635,0.409116
max,1.000000,0.936277


In [7]:
ts.loc[(ts["inv_msa_match"] == False), ['sim_ldavecs_tp_cite_10', 'sim_docvecs_tp_cite_10']].describe()

,sim_ldavecs_tp_cite_10,sim_docvecs_tp_cite_10
count,225183.000000,225183.000000
mean,0.500334,0.280472
std,0.193840,0.113732
min,0.001524,-0.518259
25%,0.364512,0.209010
50%,0.499407,0.279047
75%,0.637928,0.350697
max,1.000000,0.889535
